# Gallery Example: M/Hyper/k Queue

This example demonstrates an M/Hyper/k queueing system:
- **Arrivals**: Exponential (Poisson process)
- **Service**: Coxian distribution fitted to high SCV
- **Servers**: k servers (default k=2)
- **Capacity**: Infinite
- **Scheduling**: FCFS

The high variability service with multiple servers shows how added capacity can mitigate service variability.

In [ ]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_mhypk(k=2):
    """Create M/Hyper/k queueing model"""
    model = Network(f'M/Hyper/{k}')
    
    # Block 1: nodes
    source = Source(model, 'mySource')
    queue = Queue(model, 'myQueue', SchedStrategy.FCFS)
    sink = Sink(model, 'mySink')
    
    # Block 2: classes
    oclass = OpenClass(model, 'myClass')
    source.set_arrival(oclass, Exp(1))
    # Coxian service with mean=0.5 and SCV=4 (high variability)
    queue.set_service(oclass, Coxian.fit_mean_and_scv(0.5, 4))
    queue.set_number_of_servers(k)
    
    # Block 3: topology
    model.link(Network.serial_routing(source, queue, sink))
    
    return model

# Create the model with k=2 servers
model = gallery_mhypk(k=2)

## About M/Hyper/k Queues

For M/Hyper/k with k=2 servers:
- **Arrival rate**: λ = 1
- **Service mean**: 0.5 (rate μ = 2 per server)
- **Service SCV**: 4 (high variability, Coxian)
- **Total capacity**: k × μ = 4
- **Utilization**: ρ = λ / (k × μ) = 1/4 = 0.25

The high service variability (SCV = 4) means:
- Some jobs get very fast service
- Others require much longer
- Multiple servers help smooth out the variability

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# CTMC Solver
solver_ctmc = CTMC(model, cutoff=15)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# Fluid Solver
solver_fluid = FLD(model)
avg_table_fluid = solver_fluid.avg_table()
print("\nFluid Solver:")
print(avg_table_fluid)